<div style="font-weight: bold;font-size:40px;background-color: #A0E8FF;padding:0 5%; color:#2F4F4F">Toxic Comments</div>

<center><img src="https://www.institutotraficoonline.com/wp-content/uploads/2021/02/haters-instituto-trafico-online-ito-roberto-gamboa.jpg" width=400></center>


Hater comments are arround us, every time in every social netwrok. Thansk to [Jigsaw](https://www.kaggle.com/c/jigsaw-toxic-severity-rating) for pomoving this kind of competition in order to has a clear internet improving our moderation methods.

In this notebook you will an approach using Navie Bayes classifier because this is a very simple model with a strong stadistic point of view.

I always try to create the best notebooks as possible, and i hope you reciebe it as well. <b>So please if you find it interesting, UPVOTE my kernel</b>
you probably know how much positive impact does it have to our work as data scientist



<h2>Imports</h2>

In [ ]:
#Basics
import numpy as np
import pandas as pd
import progressbar

#Sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer

#NLP
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer() 

#Others
import warnings
warnings.filterwarnings("ignore")
import sys
sys.setrecursionlimit(5000)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<h2>Upload Datasets</h2>

Note that the train data is not provided in this competition, but jigsaw provided information about sentiment in texts in another competition before.

In [ ]:
TRAIN_DATA_PATH = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
TEST_DATA_PATH = "/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv"
VALID_DATA_PATH = "../input/jigsaw-toxic-severity-rating/validation_data.csv"
SAMPLE_SUBMISSION = "/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv"

In [ ]:
df_train = pd.read_csv(TRAIN_DATA_PATH)
df_validation_data = pd.read_csv(VALID_DATA_PATH)
df_sample_submission = pd.read_csv(SAMPLE_SUBMISSION)

<h2>Clean text</h2>

The most important step in machine learnign process, is to clean de data because "Garbage in, garbage out".


As this [post recomend](https://medium.com/analytics-vidhya/text-cleaning-in-natural-language-processing-nlp-bea2c27035a6), this clean data will include:

1. Removing HTML characters,ASCII
2. Convert Text to Lowercase
3. Remove Punctuation's
4. Remove Stop words
5. Stemming

1, 2 and 3 are included in the clean_text function. On the other hand 4 and 5 both are included in stopword function

In [ ]:
def clean_text(text):
#will replace the html characters with " "
    text=re.sub('<.*?>', ' ', text)  
    #To remove the punctuations
    text = text.translate(str.maketrans(' ',' ',string.punctuation))
    #will consider only alphabets and numerics
    text = re.sub('[^a-zA-Z]',' ',text)  
    #will replace newline with space
    text = re.sub("\n"," ",text)
    #will convert to lower case
    text = text.lower()
    # will split and join the words
    text=' '.join(text.split())
    return text

def stopwords(input_text, stop_words):
    word_tokens = word_tokenize(input_text) 
    output_text = [w for w in word_tokens if not w in stop_words]
    output = [] 
    for w in word_tokens: 
        if w not in stop_words:
            output.append(w)
            
    text = ' '.join(output)
    return text

In [ ]:
unrelevant_words = ['wiki','wikipedia','page']
#Clean step 1, 2 and 3
df_train['comment_text'] = df_train['comment_text'].apply(lambda x: ''.join([w for w in clean_text(x) if w not in unrelevant_words]))

#Clean Step 4
df_train['comment_text'] = df_train['comment_text'].apply(lambda x: ''.join([w for w in stopwords(x,stop_words)]))

#Clean Step 5
df_train['comment_text'] = df_train['comment_text'].apply(lambda x: ''.join([w for w in lemmatizer.lemmatize(x)]))

df_train.head(3)

In [ ]:
#Clean step 1, 2 and 3
df_validation_data['less_toxic'] = df_validation_data['less_toxic'].apply(lambda x: ''.join([w for w in clean_text(x) if w not in unrelevant_words]))
df_validation_data['more_toxic'] = df_validation_data['more_toxic'].apply(lambda x: ''.join([w for w in clean_text(x) if w not in unrelevant_words]))

#Clean Step 4
df_validation_data['less_toxic'] = df_validation_data['less_toxic'].apply(lambda x: ''.join([w for w in stopwords(x,stop_words)]))
df_validation_data['more_toxic'] = df_validation_data['more_toxic'].apply(lambda x: ''.join([w for w in stopwords(x,stop_words)]))

#Clean Step 5
df_validation_data['less_toxic'] = df_validation_data['less_toxic'].apply(lambda x: ''.join([w for w in lemmatizer.lemmatize(x)]))
df_validation_data['more_toxic'] = df_validation_data['more_toxic'].apply(lambda x: ''.join([w for w in lemmatizer.lemmatize(x)]))

df_validation_data.head(3)

# Scoring training data

The data provided in this contest does not include training data, so it is important to find another data source. This needs to be transformed into a score to create a toxicity scale.

Also, clases are desbalanced, thats why some taks need to be done.

In [ ]:
# Create a score that messure how much toxic is a comment
cat_mtpl = {'obscene': 0.10, 'toxic': 0.20, 'threat': 0.5, 
            'insult': 0.62, 'severe_toxic': 0.7, 'identity_hate': 1}

for category in cat_mtpl:
    df_train[category] = df_train[category] * cat_mtpl[category]

df_train['score'] = df_train.loc[:, 'toxic':'identity_hate'].sum(axis=1)

In [ ]:
#Clases aren't balanced, that's whya a balance is needed
#Balance the clases
n_samples_toxic = len(df_train[df_train['score'] != 0])
n_samples_normal = len(df_train) - n_samples_toxic

idx_to_drop = df_train[df_train['score'] == 0].index[n_samples_toxic//5:]
df_train = df_train.drop(idx_to_drop)

print(f'Reduced number of neutral text samples from {n_samples_normal} to {n_samples_toxic//5}.')
print(f'Total number of training samples: {len(df_train)}')


In [ ]:
#Feature engineering for creating 
df_tragets = pd.DataFrame(pd.unique(df_train['score'].values), columns=['target_value']).sort_values(by='target_value', ascending = True).reset_index(drop=True)
THRESHOLD = df_tragets['target_value'].quantile(q=0.2)
df_train['sentiment'] = df_train['score'].map(lambda x: 1 if x < THRESHOLD else 2 if x < THRESHOLD*2 else 3 if x < THRESHOLD*3 else 4 if x < THRESHOLD*4 else 5)

df_train = df_train[['comment_text','sentiment']].reset_index(drop=True)
df_train

## TF-IDF

[Term frequency – Inverse document frequency](https://es.wikipedia.org/wiki/Tf-idf) is one of the most popular techniques used in the land of NLP to represent words by numbers. Thats just for one simple reason: **COMPUTERS ONLY UNDERSTAND NUMBERS.**

If you want to dive into this concept, just type tfidf on google search to find a watherfalls of posts about this topics.

In [ ]:
tf_idf_vect = TfidfVectorizer(analyzer='word',stop_words= 'english')
X = tf_idf_vect.fit_transform(df_train['comment_text'])
X

# Naive Bayes

Naive Bayes is an approach selected becaouse of its well balance between fast modeling and competitive results.

The same as tf idf, just type this concept and lot of knwoladge will come to your eyes.

In [ ]:
model = MultinomialNB()
model.fit(X, df_train['sentiment'])

## Validate

This is very usefull because is a valid approach about the performance of this model.

In [ ]:
X_less_toxic = tf_idf_vect.transform(df_validation_data['less_toxic'])
X_more_toxic = tf_idf_vect.transform(df_validation_data['more_toxic'])

In [ ]:
p1 = model.predict_proba(X_less_toxic)
p2 = model.predict_proba(X_more_toxic)

In [ ]:
(p1[:, 1] < p2[:, 1]).mean()

# Predictions and load submission.csv

In [ ]:
df_test = pd.read_csv(TEST_DATA_PATH)

#Clean step 1, 2 and 3
df_test['text'] = df_test['text'].apply(lambda x: ''.join([w for w in clean_text(x) if w not in unrelevant_words]))

#Clean Step 4
df_test['text'] = df_test['text'].apply(lambda x: ''.join([w for w in stopwords(x,stop_words)]))

#Clean Step 5
df_test['text'] = df_test['text'].apply(lambda x: ''.join([w for w in lemmatizer.lemmatize(x)]))

df_test.head(3)

In [ ]:
X_test = tf_idf_vect.transform(df_test['text'])
p3 = model.predict_proba(X_test)

df_test['score'] = p3[:, 1]
df_test = df_test[['comment_id','score']]

In [ ]:
df_test

In [ ]:
df_test.to_csv('./submission.csv', index=False)